In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

2025-01-28 19:52:27.953748: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738090347.972654    9580 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738090347.980539    9580 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-28 19:52:28.010564: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
# Define a T-Net block
def t_net(inputs, k):
    x = layers.Conv1D(64, 1, activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1D(128, 1, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1D(1024, 1, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.GlobalMaxPooling1D()(x)
    
    x = layers.Dense(512, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Dense(k * k, weights=[tf.zeros([256, k * k]), tf.eye(k).flatten()])(x)
    x = layers.Reshape((k, k))(x)
    return x

# Define the PointNet model for segmentation
# Define the PointNet model for segmentation with dynamic input size
def pointnet_segmentation_dynamic(num_classes):
    inputs = layers.Input(shape=(None, 3))  # Allow variable number of points

    # Input transformation
    tnet = t_net(inputs, 3)
    transformed_inputs = tf.linalg.matmul(inputs, tnet)

    x = layers.Conv1D(64, 1, activation='relu')(transformed_inputs)
    x = layers.BatchNormalization()(x)
    
    # Feature transformation
    tnet_feature = t_net(x, 64)
    transformed_features = tf.linalg.matmul(x, tnet_feature)

    x = layers.Conv1D(64, 1, activation='relu')(transformed_features)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1D(128, 1, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1D(1024, 1, activation='relu')(x)
    x = layers.BatchNormalization()(x)

    # Global feature vector
    global_feature = layers.GlobalMaxPooling1D()(x)
    global_feature = layers.RepeatVector(tf.shape(inputs)[1])(global_feature)  # Match the number of points

    # Concatenate global and local features
    x = layers.Concatenate()([transformed_features, global_feature])

    # Fully connected layers for segmentation
    x = layers.Conv1D(512, 1, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1D(256, 1, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1D(128, 1, activation='relu')(x)
    x = layers.BatchNormalization()(x)

    outputs = layers.Conv1D(num_classes, 1, activation='softmax')(x)

    return models.Model(inputs, outputs)

# Example usage
num_classes_segmentation = 50
segmentation_model_dynamic = pointnet_segmentation_dynamic(num_classes_segmentation)
segmentation_model_dynamic.summary()
